In [1]:
from generator import Task, LinearModel
from tokenizer import FloatTokenizer
from dataset import LinearModelDataset
from config import getConfig
from train import trainModel

In [2]:
task = Task(n_predictors=2)
lm = LinearModel(task)
tokenizer = FloatTokenizer()

In [3]:
cfg = getConfig()
print(cfg)

{'n_predictors': 2, 'n_samples': 50, 'n_draws': 256, 'batch_size': 4, 'n_epochs': 10, 'lr': 0.001, 'd_model': 128, 'd_ff': 256, 'n_heads': 4, 'n_blocks_e': 2, 'n_blocks_d': 6, 'dropout': 0.1, 'seq_len': 2000, 'seq_len_out': 14, 'precision': 4, 'preload': None, 'model_folder': 'weights', 'model_basename': 'tmodel', 'experiment_name': 'runs/linear'}


In [4]:
# train_dl, val_dl, tokenizer = getDataset(cfg)

In [ ]:
trainModel(cfg)

Device: cpu
Train size: 230, Validation size: 26


Epoch 00: 100%|███████████████████████████████████████████████████████████████████| 58/58 [10:38<00:00, 11.00s/it, loss=4.38]


Error decoding: ['[SOS]', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+']
--------------------------------------------------------------------------------
Target: tensor([-0.7844,  1.4195, -0.1667,  0.1000])
Predicted: [SOS]+E-4+E-4+E-4+E-4+E-4+E-4+
Predicted (tokens): ['[SOS]', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+']
Error decoding: ['[SOS]', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+']
--------------------------------------------------------------------------------
Target: tensor([0.7648, 2.4529, 1.1117, 0.1000])
Predicted: [SOS]+E-4+E-4+E-4+E-4+E-4+E-4+
Predicted (tokens): ['[SOS]', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+', 'E-4', '+']


Epoch 01: 100%|███████████████████████████████████████████████████████████████████| 58/58 [10:50<00:00, 11.21s/it, loss=3.61]


Error decoding: ['[SOS]', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+']
--------------------------------------------------------------------------------
Target: tensor([-1.0756, -0.5996, -0.2111,  0.1000])
Predicted: [SOS]+N1000E-4+N1000E-4+N1000E-4+N1000E-4+
Predicted (tokens): ['[SOS]', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+']
Error decoding: ['[SOS]', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+']
--------------------------------------------------------------------------------
Target: tensor([ 0.1294, -0.5817,  2.1366,  0.1000])
Predicted: [SOS]+N1000E-4+N1000E-4+N1000E-4+N1000E-4+
Predicted (tokens): ['[SOS]', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+', 'N1000', 'E-4', '+']


Epoch 02: 100%|███████████████████████████████████████████████████████████████████| 58/58 [11:02<00:00, 11.42s/it, loss=3.11]


--------------------------------------------------------------------------------
Target: tensor([ 0.1539, -1.7573,  0.0880,  0.1000])
Predicted: [0.1]
Predicted (tokens): ['[SOS]', '+', 'N1000', 'E-4', '[EOS]']
--------------------------------------------------------------------------------
Target: tensor([-0.1707,  1.4741,  0.2748,  0.1000])
Predicted: [0.1]
Predicted (tokens): ['[SOS]', '+', 'N1000', 'E-4', '[EOS]']


Epoch 03:  12%|████████▏                                                           | 7/58 [01:22<10:15, 12.07s/it, loss=3.22]